In [1]:
import csv
import PIL
import matplotlib.pyplot as plt

In [2]:
def plot_image(image, ax, date, station) -> None:
    """
    display the raw image

    Args:
        image (PIL.Image.Image): opened image
        ax1 (plt.Axes): subplot axis
    """
    ax.imshow(image, aspect="auto")
    ax.set_title(
            f"Model Labeled as: {row[1]}, {row[2]}, {station}, {date}\n"
        )
    ax.axis("off")


In [3]:
date = '2022/08/06'
pred_dir = '/ai2es/realtime_predictions/csv'
pred = 'precipitation'
with open(f"{pred_dir}/{date}/{date.replace('/','_')}.csv", newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    # reader.__next__() # skip header
    for row in reader:
        if row[1] == pred:
            try:
                station = row[0].split('.')[0].split('_')[1]
                image = PIL.Image.open(f'/ai2es/cam_photos/{date}/{station}/{row[0]}')
            except FileNotFoundError:
                print("The file cannot be found.")
            _, ax = plt.subplots(ncols=1, nrows=1)
            plot_image(image, ax, date, station)
            plt.show()
        